#Ready to publish! 
just add notes on how t use 

In [ ]:
#pip install spotipy
from typing import Set
import spotipy
from spotipy.oauth2 import SpotifyOAuth 
import numpy as np
import math

def spiderweb(max,list_of_artist,level,results):
  
  if level==max:
    return
  
  else:
    for id in list_of_artist:
      related_artist=sp.artist_related_artists(artist_id=id)
      for uri in related_artist['artists']:
        if uri['uri'] not in results:
          results.append(uri['uri'])
    list_of_artist.extend(results[:])
    return spiderweb(max,artist,level+1,results)

def breakdown(list_of_artist):
    if len(list_of_artist)<2:
      for uri in list_of_artist:
        songs[1]=sp.artist_top_tracks(artist_id=uri)
        for song in songs[1]['tracks']:
            songs[2].append(song['uri'])
      return
    else:
      half=len(list_of_artist)*0.5
      half=math.ceil(half)
      other_half=half
      breakdown(list_of_artist[:half])
      breakdown(list_of_artist[other_half:])


def similarity (seed,top_songs,related_songs):
  seed_features=sp.audio_features(seed)
  seed_features=seed_features[0]
  length=len(top_songs)
  rem=length%100
  length=(length-rem)/100
  if rem>0:
    length+=1
  uri=[]
  for iter in range(int(length)):
    uri=top_songs[:100]
    top_song_features=sp.audio_features(uri)
    del top_songs[:100]
    uri.clear()
    for track_info in top_song_features:
      difference=total=0
      denom=len(attributes)
      error=0
      for y in attributes:
        try:
          difference=abs(seed_features[y]-track_info[y])
        except TypeError:
            denom-=1
            error=weights[y]/denom
            break
        
        difference+=1
        if difference!=0:
          difference=1/difference
          total+=(difference*(weights[y]+error))
        else:
          denom-=1    
      total=round(total,2)
      if total>=0.9 and total!=1:
        related_songs.append(track_info['uri'])
#*****************your info ************************
SPOTIPY_CLIENT_ID  = 'your client id'
SPOTIPY_CLIENT_SECRET = 'your client secret'
SPOTIPY_REDIRECT_URI= 'https://google.com/'
username= 'your username'
scope = "user-library-read playlist-modify-public playlist-modify-private playlist-read-private"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(SPOTIPY_CLIENT_ID,SPOTIPY_CLIENT_SECRET,SPOTIPY_REDIRECT_URI,username,scope=scope,open_browser=False))

#*****************weights************************

weights={'danceability':0.13725490196078433 ,'energy':0.09803921568627451,'mode':0.0392156862745098,'speechiness':0.1568627450980392,'acousticness':0.09803921568627451,'instrumentalness':0.1568627450980392,'liveness':0.1568627450980392 ,'valence':0.0784313725490196,'tempo':0.0784313725490196}
attributes=['danceability','energy','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo'] 


# main program*************************************************************

flag=input("Welcome to the playlist builder that finds similar songs to the ones inputted\n\nEnter 0 to create a new playlist\nEnter 1 to add to an existing playlist you created\n")
if flag=='0':
  name=input('Input the name of the playlist\n')
  playlist_name=sp.user_playlist_create(username,name)
  playlist_name=playlist_name['id']
  flag=3
elif flag=='1':
  text=input('Enter the playlist link here\n')
  link,playlist_name=text.split("playlist/")
  flag=3
else:
  print('Invalid entry exiting program')
while flag==3:
  songs=[[],[],[],[]]
  artist=[]
  song_name=input('Please enter a song and the artist\'s name\n\n')
  results=sp.search(q=song_name,type='track,artist')
  try:
    seed_artist=results['tracks']['items'][0]['artists'][0]['uri']
  except:
    print('Invalid entry or no results found\n')
  else:
    seed_track=results['tracks']['items'][0]['uri']
    artist.append(seed_artist)
    spiderweb(max=2,list_of_artist=artist,level=0,results=songs[0])
    breakdown(list(set(artist)))
    similarity (seed_track,songs[2],songs[3])  
    final_songs=list(set(songs[3]))
    np.random.shuffle(final_songs)
    length=len(final_songs)
    rem=length%100
    length=(length-rem)/100
    if rem>0:
      length+=1
    for iter in range(int(length)):
      add=final_songs[:100]
      sp.playlist_add_items(playlist_id=playlist_name, items=add)
      del final_songs[:100]
      add.clear()
    exit=input("If you would like to add another song enter 'y' or press enter\n")
    if exit=='y':
      pass
    else: break
if flag==3:
  print('\nHere is your playlist link \nhttps://open.spotify.com/playlist/'+playlist_name)